In [1]:
import pandas as pd

data = pd.read_csv('quora_question_pairs.csv')
duplicates = data[data['is_duplicate'] == 1]
filtered_data = duplicates[['question1', 'question2']]


FileNotFoundError: [Errno 2] No such file or directory: 'quora_question_pairs.csv'

In [ ]:
exploratory_starters = ['What', 'How', 'Why', 'Explain']

def is_exploratory(question):
    return any(question.strip().startswith(starter) for starter in exploratory_starters)

filtered_data = filtered_data[
    filtered_data['question1'].apply(is_exploratory) &
    filtered_data['question2'].apply(is_exploratory)
]


In [ ]:
annotated_data = filtered_data.rename(columns={
    'question1': 'Query',
    'question2': 'Suggested_Followup'
})
annotated_data['Context'] = ""

annotated_data.to_csv('annotated_conversational_data.csv', index=False)


In [ ]:
small_dataset = annotated_data.sample(50, random_state=42)
small_dataset.to_csv('small_annotated_data.csv', index=False)


# TF-IDF
### Start from here

In [1]:
import pandas as pd

data = pd.read_csv('annotated_conversational_data.csv')
queries = data['Query'].tolist()
suggestions = data['Suggested_Followup'].tolist()

documents = queries + suggestions


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
tfidf_matrix = vectorizer.fit_transform(documents)


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

def suggest_queries(input_query, vectorizer, tfidf_matrix, documents, top_n=3):
    """
    Suggest follow-up queries based on an input query.

    Args:
        input_query (str): The initial query input by the user.
        vectorizer (TfidfVectorizer): The fitted TF-IDF vectorizer.
        tfidf_matrix (sparse matrix): TF-IDF matrix of the documents.
        documents (list): List of documents (queries and suggestions).
        top_n (int): Number of suggestions to return.

    Returns:
        list: Ranked list of suggested queries.
    """
    input_vector = vectorizer.transform([input_query])
    similarities = cosine_similarity(input_vector, tfidf_matrix).flatten()
    ranked_indices = similarities.argsort()[::-1][:top_n]

    return [documents[i] for i in ranked_indices if documents[i] != input_query][:top_n]


# Test follow-up queries

In [4]:
input_query = "What are the benefits of yoga?"
suggested_queries = suggest_queries(input_query, vectorizer, tfidf_matrix, documents)

print("Input Query:", input_query)
print("Suggested Follow-up Queries:")
for i, suggestion in enumerate(suggested_queries, 1):
    print(f"{i}. {suggestion}")


Input Query: What are the benefits of yoga?
Suggested Follow-up Queries:
1. How do you do yoga?
2. What is yoga all about?
3. What is yoga for?


In [5]:
def precision_at_k(relevant, retrieved, k):
    return len(set(relevant) & set(retrieved[:k])) / k

relevant_suggestions = ["How does yoga improve health?", "What are the mental benefits of yoga?"]
precision = precision_at_k(relevant_suggestions, suggested_queries, k=3)
print("Precision@3:", precision)


Precision@3: 0.0


# Simple Keyword Extraction

In [7]:
def simple_keyword(query, n=10):
    corpus = [query]
    # Compute TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(corpus)
    
    # Get keywords from the query
    keywords = vectorizer.get_feature_names_out()
    scores = X.toarray()[0]
    keyword_scores = list(zip(keywords, scores))
    sorted_keywords = sorted(keyword_scores, key=lambda x: x[1], reverse=True)
    
    return sorted_keywords[:n]

simple_keyword(input_query)

[('benefits', 0.7071067811865475), ('yoga', 0.7071067811865475)]

In [10]:
data['keyword_1'] = [''] * len(data)

for i in range(len(data)):
    try:
        data.loc[i,'keyword_1'] = ', '.join([k for k, v in simple_keyword(data.loc[i,'Suggested_Followup'])]) # + " " + data.loc[i,'Query'])])
    except:
        continue

data.head(5)

,Query,Suggested_Followup,Context,keyword_1
0,How can I be a good geologist?,What should I do to be a great geologist?,NaN,"geologist, great"
1,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,NaN,"comments, youtube"
2,What can make Physics easy to learn?,How can you make physics easy to learn?,NaN,"easy, learn, make, physics"
3,What was your first sexual experience like?,What was your first sexual experience?,NaN,"experience, sexual"
4,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,NaN,"affect, planning, presently, presidency, stude..."


# Intermediate Keyword Extraction

In [11]:
!pip install rake-nltk

In [12]:
from rake_nltk import Rake

def rake_keyword(query):
    # Use RAKE to extract keywords
    r = Rake()
    r.extract_keywords_from_text(query)
    keywords = r.get_ranked_phrases()
    return set(keywords)

rake_keyword(input_query)

{'benefits', 'yoga'}

In [13]:
data['keyword_2'] = [''] * len(data)

for i in range(len(data)):
    try:
        data.loc[i,'keyword_2'] = ', '.join(rake_keyword(data.loc[i,'Suggested_Followup'])) # + " " + data.loc[i,'Query']))
    except:
        continue

data.head(5)

,Query,Suggested_Followup,Context,keyword_1,keyword_2
0,How can I be a good geologist?,What should I do to be a great geologist?,NaN,"geologist, great",great geologist
1,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,NaN,"comments, youtube","youtube comments, see"
2,What can make Physics easy to learn?,How can you make physics easy to learn?,NaN,"easy, learn, make, physics","make physics easy, learn"
3,What was your first sexual experience like?,What was your first sexual experience?,NaN,"experience, sexual",first sexual experience
4,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,NaN,"affect, planning, presently, presidency, stude...","trump presidency affect, study, planning, stud..."
